Retreve data from a storage at The Things Network and plot the main results.

In [ ]:
# data management
import requests
import json
import pandas as pd
import numpy as np

In [ ]:
# plotting -- add nice date formatter for x-axis
from matplotlib import pyplot as plt
from datetime import datetime, timedelta, date
import matplotlib.dates as mdates
import matplotlib.units as munits
converter = mdates.ConciseDateConverter()
munits.registry[np.datetime64] = converter
munits.registry[date] = converter
munits.registry[datetime] = converter

Now, it is time to get the actual data. We need to do a HTTP request, and to be authorized to do so.

In [ ]:
# name of the app on TTN
app = 'glos-office-sensing'

# we have four temperature sensors at Tiller
devices = ['eui-a840416781831be1']

# authorization key (should have been hidden, but this is just for reading, so, well, let's say it's ok, this time!)
akey = 'NNSXS.CEETDSAFAL2A4UM4N5DH4N5ASU2RHUVWBTXIQDA.ELMAAXUVEISX4DVJGL2EJLXKLAFGINZEAQOJVEW2EX5F6ETJAO2Q'

In [ ]:
# URL for requesting all data
url = f'https://eu1.cloud.thethings.network/api/v3/as/applications/{app}/packages/storage/'

# how to get data from just one unit, in this case, the first
url0 = f'https://eu1.cloud.thethings.network/api/v3/as/applications/{app}/devices/{devices[0]}/packages/storage/'

# headers, for the http GET request
headers = {
    'Accept': 'text/event-stream', 
    'Authorization': f'Bearer {akey}',
}
params =  {
    # 'after': '2021-09-21T00:00:00Z',  # just receive after this
    # 'field_mask': 'up.uplink_message.decoded_payload',  # just receive this info
}

In [ ]:
# let's make the actual request, and hope for response code 200 -- OK!
r = requests.get(url, headers=headers, params=params)
r

The data is stored/retrieved in 'r.text' as a list of json-objects. Let us now load this as more manageable data.

In [ ]:
def parse_response(r):
    # parse the response and return a list of json-loads-objects (dicts)
    # we filter out lines that have no info, and then those with no decoded payload
    out = [json.loads(line) for line in r.text.splitlines() if len(line)]
    out = [item for item in out if 'decoded_payload' in item['result']['uplink_message']]
    return out

In [ ]:
# let's parse the results, and see if we get something meaningful
p = parse_response(r)
# print(p[0])  # uncomment this to see the datastructure!
print(p[0]['result']['end_device_ids']['device_id'])
print(len(p))

Json-dicts are neat for look up of individual data, but not so nice for direct plotting. Let's define some functions to filter out data and make us some lists, that are easier to handle. We take one data point from each json object in the list 'p'.

In [ ]:
def p2devID(p):
    # return the device id for each item 
    # p = parse_response(r)
    return [item['result']['end_device_ids']['device_id'] for item in p]

def p2payload(p):
    # return the decoded payload for each item 
    return [item['result']['uplink_message']['decoded_payload'] for item in p]
    
def p2rssi(p, datano=0):
    # return the signal strength for each item 
    return [item['result']['uplink_message']['rx_metadata'][datano]['rssi'] for item in p]
        
def p2time(p):
    # return the received time for each item 
    return [pd.to_datetime(item['result']['received_at']) for item in p]

In [ ]:
# we can now test the functions, to see if the results are satisfactory
print(p2devID(p)[0])
print(p2payload(p)[0])
print(p2rssi(p)[0])
print(p2time(p)[0])

print('Are all of equal lenght?')  # using funky python chaning of operators here! :D
print(len(p2devID(p)) == len(p2payload(p)) == len(p2rssi(p)) == len(p2time(p)))

Now that we can easily make lists, we can easily make a dataframe, which enables easier manipulation of data!

In [ ]:
pl = p2payload(p)
df = pd.DataFrame({
    'devID': p2devID(p),
    'payload': pl,
    'Temp1': [i['TempC_SHT'] for i in pl],
    'Hum1': [i['Hum_SHT'] for i in pl],
    'BatV': [i['BatV'] for i in pl],
    'rssi': p2rssi(p),
    'time': p2time(p),
})
df['time'] = pd.to_datetime(df['time'])  # need to be explicit about this, for the plotter to understand later
df = df.set_index('time')  # having the date as inxed enables us to plot a series, without passing this explicitly

In [ ]:
# this is how our dataframe now looks
df

So far, so good! We have asked for, received, parsed, filtered, and kneaded our data into a nice format. This was all done to facilitate plotting of the data. We will inspect the **temperature**, which was the main point, but also look at **signal strength** and **battery voltage**.

In [ ]:
def plotter(what='Temp1', ylabel=None, left='2021-01-01', right=None):
    # create a plot from the data in df
    dfp = df.copy()
    
    if left is not None:  # define where the plotting starts
        left = pd.Timestamp(left, tz="Europe/Brussels")
        dfp = df.loc[df.index > left]
    if right is not None:  # define where the plotting ends
        right = pd.Timestamp(right, tz="Europe/Brussels")
        dfp = df.loc[df.index < right]

    plt.figure(dpi=200)  # use 400 dpi to get a larger figure
    for devID in sorted(set(dfp['devID'])):              # reduce the list of devID to a set, and sort it
        data = dfp.loc[dfp['devID'] == devID][what]  # filter data for a given dev ID
        plt.plot(data, label=devID)  # plot line with correct label
    plt.legend()
    plt.ylabel(ylabel)

In [ ]:
plotter(what='Temp1', ylabel='Grader celsius')

In [ ]:
plotter(what='rssi', ylabel='Signalstyrke')

In [ ]:
plotter(what='BatV', ylabel='Batteristyrke')

... and that was the intitial test. Let's plot it all together!

In [ ]:
def htplotter(left='2021-01-01', right=None):
    # create a plot from the data in df
    dfp = df.copy()
    
    if left is not None:  # define where the plotting starts
        left = pd.Timestamp(left, tz="Europe/Brussels")
        dfp = df.loc[df.index > left]
    if right is not None:  # define where the plotting ends
        right = pd.Timestamp(right, tz="Europe/Brussels")
        dfp = df.loc[df.index < right]


    fig, ax1 = plt.subplots(dpi=200)  # use 400 dpi to get a larger figure
    ax2 = ax1.twinx()
    ax2._get_lines.prop_cycler = ax1._get_lines.prop_cycler  # use same cycler for both axes
     
    for devID in sorted(set(dfp['devID'])):              # reduce the list of devID to a set, and sort it
        data = dfp.loc[dfp['devID'] == devID]['Temp1']   # filter data for a given dev ID
        ax1.plot(data, ls='-.', label=f'Temp {devID}')  # plot line with correct label
        data = dfp.loc[dfp['devID'] == devID]['Hum1']   # filter data for a given dev ID
        ax2.plot(data, ls='--', label=f'Hum {devID}')  # plot line with correct label
    fig.legend()
    ax1.set_ylabel('Temperature')
    ax2.set_ylabel('Humidity')

In [ ]:
htplotter()